In [ ]:
## Imports
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import shutil
import csv
import yaml

In [ ]:
with open('config.yaml') as stream:
    config = yaml.safe_load(stream)

print(config['appname'])
print(config['options'])


In [ ]:
def prepareFolder(path):
    if os.path.exists(path+'/') == True:
        shutil.rmtree(path)
    if os.path.exists(path+'/') == False:
        os.makedirs(path+'/')

In [ ]:
## Constants
COLORS = ['#1f77b4',
          '#ff7f0e',
          '#2ca02c',
          '#d62728',
          '#9467bd',
          '#8c564b',
          '#e377c2',
          '#7f7f7f',
          '#bcbd22',
          '#17becf',
          '#1a55FF']
CAT = {
    'usr': 'total-cpu-usage',
    'sys': 'total-cpu-usage',
    'idl': 'total-cpu-usage',
    'wai': 'total-cpu-usage',
    'hiq': 'total-cpu-usage',
    'siq': 'total-cpu-usage',

    'used' : 'memory usage',
    'buff' : 'memory usage',
    'cach' : 'memory usage',
    'free' : 'memory usage',

    'files': 'filesystem',
    'inodes' : 'fileystem',

    'read' : 'dsk/total',
    'writ' : 'dsk/total',
    'reads' : 'dsk/total',
    'writs' : 'dsk/total',

    'recv' : 'net/eth0',
    'send' : 'net/eth0',

    'lis' : 'tcp sockets',
    'act' : 'tcp sockets',
    'syn' : 'tcp sockets',
    'tim' : 'tcp sockets',
    'clo' : 'tcp sockets',

    'tot': 'sockets',
    'tcp': 'sockets',
    'udp': 'sockets',
    'raw': 'sockets',
    'frg': 'sockets',

    'int' : 'system',
    'csw' : 'system',

    'run' : 'procs',
    'blk' : 'procs',
    'new' : 'procs'
}

CAT_NAMES = ['total-cpu-usage', 'memory-usage', 'filesystem', 'dsk-total1', 'dsk-total2', 'net-eth0', 'tcp sockets', 'sockets', 'system', 'procs',]

COL_NAMES_DOCUMENTATION_DO_NOT_DELETE = [
    # total-cpu-usage
    'usr: CPU usage by user processes [%]', 
    'sys: CPU usage by system processes [%]',
    'idl: Number of idle processes [#]',
    'wai: Number of waiting processes [#]',
    'hiq: Number of hard interrupts [#]',
    'siq: Number of soft interrupts [#]',
    # memory usage
    # https://www.tecmint.com/dstat-monitor-linux-server-performance-process-memory-network/
    # https://linuxtect.com/linux-dstat-command-monitor-linux-performance-and-resource-usage/
    # https://www.thegeekdiary.com/understanding-proc-meminfo-file-analyzing-memory-utilization-in-linux/
    # /proc/meminfo
    'used: Amount of used memory [Bytes]', # / 1024 = KB
    'buff: Amount of buffered memory [Bytes]', # / 1024 / 1024 = MB
    'cach: Amount of cached memory [Bytes]',
    'free: Amount of free memory [Bytes]',

    # filesystem
    # https://www.linuxquestions.org/questions/linux-general-1/proc-sys-fs-file-nr-57646/
    # /proc/sys/fs/file-nr
    'files: Number of allocated file handles [#]',
    'inodes: Number of used file handles [#]',

    # dsk/total
    'read: Amount of read bytes on disk [Bytes]', # not 100% sure
    'writ: Amount of written bytes on disk [Bytes]', # / 1024 = KB

    # dsk/total
    'reads: Number of read operations on disk [#]',
    'writs: Number of write operations on disk [#]',
    
    # net/eth0
    'recv: Amount of received bytes on eth0 [B]', # /1024 = k
    'send: Amount of received bytes on eth0 [B]',

    # tcp sockets
    # http://karunsubramanian.com/network/what-is-syn_sent-socket-status/
    # /proc/net/tcp
    # /proc/net/tcp6
    'lis: Number of TCP connections with status "listening" [#]', 
    'act: Number of TCP connections with status "established" (active) [#]',
    'syn: Number of TCP connections with status "syn_sent", "syn_receive" or "last_ack" [#]',
    'tim: Number of TCP connections with status "waiting"[#]',
    'clo: Number of TCP connections with status "fin-wait1/2", "close/_wait" or "closign" (closed) [#]',

    # sockets
    # https://community.hpe.com/t5/Networking/Regarding-meaning-for-quot-cat-proc-net-sockstat-quot-results/td-p/3922867#.Yvnld3VBxhE
    # /proc/net/sockstat
    'tot: Number of total sockets [#]',
    'tcp: Number of tcp sockets [#]',
    'udp: Number of udp sockets [#]',
    'raw: Number of raw (using no protocol) sockets [#]',
    'frg: Number of frag sockets [#]',

    # system
    # https://www.tecmint.com/dstat-monitor-linux-server-performance-process-memory-network/
    'int: Number of interrupts [#]', 
    'csw: Number of ontext switches [#]', 

    # procs
    # /proc/stat
    'run: Number of processes with status "running" [#]', 
    'blk: Number of processes with status "blocked" [#]',
    'new: Number of processes with status "new" [#]' 
]

COL_NAMES = [
    # total-cpu-usage
    'usr: CPU usage by user processes [%]', 
    'sys: CPU usage by system processes [%]',
    'idl: Number of idle processes [#]',
    'wai: Number of waiting processes [#]',
    'hiq: Number of hard interrupts [#]',
    'siq: Number of soft interrupts [#]',
    # memory usage
    # https://www.tecmint.com/dstat-monitor-linux-server-performance-process-memory-network/
    # https://linuxtect.com/linux-dstat-command-monitor-linux-performance-and-resource-usage/
    # https://www.thegeekdiary.com/understanding-proc-meminfo-file-analyzing-memory-utilization-in-linux/
    # /proc/meminfo
    'used: Amount of used memory [Bytes]', # / 1024 = KB
    'buff: Amount of buffered memory [Bytes]', # / 1024 / 1024 = MB
    'cach: Amount of cached memory [Bytes]',
    'free: Amount of free memory [Bytes]',

    # filesystem
    # https://www.linuxquestions.org/questions/linux-general-1/proc-sys-fs-file-nr-57646/
    # /proc/sys/fs/file-nr
    'files: Number of allocated file handles [#]',
    'inodes: Number of used file handles [#]',

    # dsk/total
    'read: Amount of read bytes on disk [Bytes]', # not 100% sure
    'writ: Amount of written bytes on disk [Bytes]', # / 1024 = KB

    # dsk/total
    'reads: Number of read operations on disk [#]',
    'writs: Number of write operations on disk [#]',
    
    # net/eth0
    'recv: Amount of received bytes on eth0 [B]', # /1024 = k
    'send: Amount of received bytes on eth0 [B]',

    # tcp sockets
    # http://karunsubramanian.com/network/what-is-syn_sent-socket-status/
    # /proc/net/tcp
    # /proc/net/tcp6
    'lis: Number of TCP connections "listening" [#]', 
    'act: Number of TCP connections "established" (active) [#]',
    'syn: Number of TCP connections "syn_s", "syn_r" or "last_a" [#]',
    'tim: Number of TCP connections with status "waiting"[#]',
    'clo: Number of TCP connections with status "closing"  [#]',

    # sockets
    # https://community.hpe.com/t5/Networking/Regarding-meaning-for-quot-cat-proc-net-sockstat-quot-results/td-p/3922867#.Yvnld3VBxhE
    # /proc/net/sockstat
    'tot: Number of total sockets [#]',
    'tcp: Number of tcp sockets [#]',
    'udp: Number of udp sockets [#]',
    'raw: Number of raw sockets [#]',
    'frg: Number of frag sockets [#]',

    # system
    # https://www.tecmint.com/dstat-monitor-linux-server-performance-process-memory-network/
    'int: Number of interrupts [#]', 
    'csw: Number of ontext switches [#]', 

    # procs
    # /proc/stat
    'run: Number of processes with status "running" [#]', 
    'blk: Number of processes with status "blocked" [#]',
    'new: Number of processes with status "new" [#]' 
]

## Create dataframes

In [ ]:
from dfLoader import main

dfs, names = main()
# sanity check
for (df, name) in zip(dfs, names):
     print (df['usr'].iloc[0], name)


In [ ]:
# Time Series
prepareFolder('plots/timeseries')
plt.style.use('classic')
for (df, name) in zip(dfs, names):
    print (df['usr'].iloc[0], name)
    os.mkdir('plots/timeseries/'+name)
    index = 0
    for column in df.columns:
        plt.figure(figsize=(21, 7), dpi=200)
        plt.hist(df[column])
        plt.subplot(2, 1, 1)
        df[column].plot(marker="o", color='C0', alpha=0.5, linewidth=0.5, label=COL_NAMES[index])
        plt.axhline(y=df[column].mean(), color='C1', alpha=1, linestyle='-', linewidth=4, label='mean')
        plt.axhline(y=df[column].max(), color='C2', alpha=1, linestyle=':', linewidth=2, label='max')
        plt.axhline(y=df[column].min(), color='C6', alpha=1, linestyle=':', linewidth=2, label='min')
        plt.title('{}'.format(CAT.get(column))) 
        plt.ylabel(column)
        plt.legend(bbox_to_anchor=(1, 0.75), loc="upper left")

        
        plt.subplot(2, 1, 2)
        df[column].plot(marker="o", color='C0', alpha=0.5, linewidth=0.5, label=COL_NAMES[index])
        plt.axhline(y=df[column].median(), color='C1', alpha=1, linestyle='-', linewidth=4, label='median')
        plt.axhline(y=df[column].quantile(q=0.75), color='C2', alpha=1, linestyle=':', linewidth=2, label='quantile (0.75)')
        plt.axhline(y=df[column].quantile(q=0.95), color='C6', alpha=1, linestyle=':', linewidth=2, label='quantile (0.95)')
        plt.ylabel(column)
        plt.legend(bbox_to_anchor=(1, 0.75), loc="upper left")
        plt.savefig('plots/timeseries/{}/{}-timeseries.png'.format(name, column), facecolor='white', transparent=False, bbox_inches="tight")
        index += 1
    plt.cla()        
plt.close('all')

In [ ]:
# Boxplot
prepareFolder('plots/boxplot')

for (df, name) in zip(dfs, names):
    os.mkdir('plots/boxplot/'+name)
    index = 0
    for column in df.columns:
        plt.figure(figsize=(10, 8), dpi=100)
        sns.boxplot(data=df[column])
        plt.ylabel(column)
        plt.legend(bbox_to_anchor=(1, 0.75), loc="upper left")
        plt.savefig('plots/boxplot/{}/{}-boxplot.png'.format(name, column), facecolor='white', transparent=False)
        plt.cla()        
plt.close('all')



In [ ]:
# Decompose
prepareFolder('plots/decompose')

for (df, name) in zip(dfs, names):
    os.mkdir('plots/decompose/'+name)
    index = 0
    for column in df.columns:
        plt.figure(figsize=(10, 8), dpi=100)
        sm.tsa.seasonal_decompose(df[column], model="add",  period=1).plot()
        plt.savefig('plots/decompose/{}/{}-decompose.png'.format(name, column), facecolor='white', transparent=False)
        plt.cla()     
plt.close('all')


In [ ]:
# Standard deviation
prepareFolder('plots/standard-deviation')

for (df, name) in zip(dfs, names):
    df_total_cpu_usage = df[['usr', 'sys', 'idl', 'wai', 'hiq', 'siq']]
    df_memory_usage = df[['used', 'buff', 'cach', 'free']]
    df_filesystem = df[['files', 'inodes']]
    df_dsk_total_1 = df[['read', 'writ']]
    df_dsk_total_2 = df[['reads', 'writs']]
    df_net_eth0 = df[['recv', 'send']]
    df_tcp_sockets = df[['lis', 'act', 'syn', 'tim', 'clo']]
    df_sockets = df[['tot', 'udp', 'raw', 'frg']]
    df_system = df[['int', 'csw']]
    df_procs = df[['run', 'blk', 'new']]

    std_categories = [df_total_cpu_usage, df_memory_usage, df_filesystem, df_dsk_total_1, df_dsk_total_2, df_net_eth0, df_tcp_sockets, df_sockets, df_system, df_procs]

    index= 0
    os.mkdir('plots/standard-deviation/'+name)
    plt.figure(figsize=(10, 8), dpi=100)
    for i in std_categories:
        stds_y = []
        stds_x = i.columns.to_list()
        for std in i.std():
            stds_y.append(std)
        sns.barplot(x=stds_x, y=stds_y)
        plt.xlabel('metrics')
        plt.ylabel('standard deviation')
        plt.title('{}'.format(CAT_NAMES[index])) 
        plt.savefig('plots/standard-deviation/{}/{}-std.png'.format(name,CAT_NAMES[index]), facecolor='white', transparent=False)
        index += 1
        plt.cla()
plt.close('all')

## Threshold

In [ ]:
with open('config.yaml') as stream:
    config = yaml.safe_load(stream)

# load dataset and names
from dfLoader import main

dfs, names = main()
# sanity check
for (df, name) in zip(dfs, names):
     print (df['usr'].iloc[0], name)
     
prepareFolder('plots/threshold')

WINDOWS = config['windows'] # number of datapoints of the non-overlapping windows
INFECTED = config['infected'] # positions of the infected datasets
HEALTHY = config['healthy'] # position of the healthy dataset
FUNCTIONS = config['functions'] # aggragation functions

# reset index (can't plot a time series)
dfs[HEALTHY[0]].reset_index(inplace=True)

# export col names for labeling
metrics = dfs[HEALTHY[0]].columns.values.tolist()
metrics = metrics[1:] # remove time col

plt.style.use('classic')
for infIndex in INFECTED:
    # reset index
    startDate = dfs[infIndex].index[0]
    dfs[infIndex].reset_index(inplace=True)
    path = 'plots/threshold/'+names[infIndex]
    folderPrefixLenght = 15 # 01csv-infected(MALWARE) from '0' to '('
    os.makedirs(path)
    for window in WINDOWS:

        dirPath = f'{path}/{window}'
        os.makedirs(dirPath)
        for function in FUNCTIONS:
            for metric in metrics:
                
                col = metric + '-' + function

                if function == 'mean':
                    dfs[HEALTHY[0]][col]  = dfs[HEALTHY[0]][metric].rolling(window).mean()[window-1::window]
                    dfs[HEALTHY[0]][col].bfill(inplace = True)
                    dfs[infIndex][col]  = dfs[infIndex][metric].rolling(window).mean()[window-1::window]
                    dfs[infIndex][col].bfill(inplace = True)
                elif function == 'max':
                    dfs[HEALTHY[0]][col]  = dfs[HEALTHY[0]][metric].rolling(window).max()[window-1::window]
                    dfs[HEALTHY[0]][col].bfill(inplace = True)
                    dfs[infIndex][col]  = dfs[infIndex][metric].rolling(window).max()[window-1::window]
                    dfs[infIndex][col].bfill(inplace = True)
                elif function == 'min':
                    dfs[HEALTHY[0]][col]  = dfs[HEALTHY[0]][metric].rolling(window).min()[window-1::window]
                    dfs[HEALTHY[0]][col].bfill(inplace = True)
                    dfs[infIndex][col]  = dfs[infIndex][metric].rolling(window).min()[window-1::window]
                    dfs[infIndex][col].bfill(inplace = True)
                else:
                    pass
                
                plt.figure(figsize=(10,6))
            
                # rolling, non-overlapping aggreation function such as mean, max and min 
                dfs[HEALTHY[0]][col].plot(marker="+", color='C0', alpha=0.5, linewidth=0.5, label= col + " healthy")
                dfs[infIndex][col].plot(marker="+", color='C2', alpha=0.5, linewidth=0.5, label= col + " infected")
                
            
                # min and max
                minHealthy = dfs[HEALTHY[0]][col].min()
                maxHealthy = dfs[HEALTHY[0]][col].max()
                minInfected = dfs[infIndex][col].min()
                maxInfected = dfs[infIndex][col].max()
                plt.axhline(y=minHealthy, color='C3', linestyle='-.', linewidth=2, label='min Healthy = {:.2f}'.format(minHealthy))
                plt.axhline(y=maxHealthy, color='C3', linestyle='-.', linewidth=2, label='max Healthy = {:.2f}'.format(maxHealthy))
                plt.axhline(y=minInfected, color='C4', linestyle='-.', linewidth=2, label='min Infected = {:.2f}'.format(minInfected))
                plt.axhline(y=maxInfected, color='C4', linestyle='-.', linewidth=2, label='max Infected = {:.2f}'.format(maxInfected))
                

                # visualize the easy thresholds
                if maxHealthy < minInfected:
                    difference = minInfected - maxHealthy
                    threshold = maxHealthy + (0.5 * difference)
                    plt.axhspan(maxHealthy, minInfected, color='C1', alpha=0.25)
                    plt.axhline(y=threshold, color='C1', linestyle='-', linewidth=3, alpha=1, label='threshold = {:.2f}'.format(threshold))
                    with open ('policy({}-{}).csv'.format(function, window),'a', newline='') as policy:
                        csv_writter = csv.writer(policy)
                        csv_writter.writerow(policyRule)

                if maxInfected < minHealthy:
                    difference = minHealthy - maxInfected
                    threshold = maxInfected + (0.5 * difference)
                    plt.axhspan(maxInfected, minHealthy, color='C1', alpha=0.25)
                    plt.axhline(y=threshold, color='C1', linestyle='-', linewidth=3, alpha=1, label='threshold = {:.2f}'.format(threshold))
                    policyRule = [names[infIndex][folderPrefixLenght:-1], col, '<=', threshold]
                    with open ('policy({}-{}).csv'.format(function, window),'a', newline='') as policy:
                        csv_writter = csv.writer(policy)
                        csv_writter.writerow(policyRule)
                
                plt.xlabel('Timestamp [hh:mm:ss] from {}'.format(startDate))
                plt.ylabel(col)
                plt.legend(bbox_to_anchor=(1, 0.75), loc="upper left")
                plt.tight_layout()
                plt.title(col + names[infIndex][5:])
                plt.savefig(path+'/'+'{}'.format(window)+'/'+col+'({})-'.format(window)+names[infIndex][6:], facecolor='white', transparent=False)
                plt.cla()
plt.close('all')